# Introduction to Deep Learning with PyTorch

 [PyTorch](http://pytorch.org/)를 활용하여 간단한 신경망 모형을 구현해 보자. Pytorch의 tensor는 앞서 학습한 Numpy의 ndarray와 매우 유사하다. Tensor는 또한 backpropagation을 위해 자동으로 gradient를 계산하는 기능을 제공한다. 

## Neural Networks

Deep Learning 은 artificial neural networks에 기반을 두고 있다. 인공 신경망 모형은 neurons으로 구성된다. 각 뉴런은 input을 받아 weight에 곱하여 이를 모두 더한다. 이를 activation function(활성화 함수)를 거쳐 최종 output을  출력한다.

![img](../assets/simple_neuron.png)

수학적 기호로 표현하면 다음과 같다.: 

$$
\begin{align}
y &= f(w_1 x_1 + w_2 x_2 + b) \\
y &= f\left(\sum_i w_i x_i +b \right)
\end{align}
$$

이것은 두 vectors의 dot/inner product 로 표현할 수 있다:

$$
h = \begin{bmatrix}
x_1 \, x_2 \cdots  x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_1 \\
           w_2 \\
           \vdots \\
           w_n
\end{bmatrix}
$$

## Tensors

-  vector 는 1-dimensional tensor
-  matrix is a 2-dimensional tensor
-  3-dimensional tensor

텐서는 neural networks의 기본적인 자료 구조이다. 

![img](../assets/tensor_examples.svg)

기본을 익히기 위해 tensor를 사용해서 간단한 neural network를 구성하는 실습을 해보자.

In [1]:
# First, import PyTorch
import torch

In [1]:
def activation(x):
    """ Sigmoid activation function 
    
        Arguments
        ---------
        x: torch.Tensor
    """
    return 1/(1+torch.exp(-x))

In [3]:
### Generate some data
torch.manual_seed(7) # Set the random seed so things are predictable

# Features are 5 random normal variables
features = torch.randn((1, 5))
# True weights for our data, random normal variables again
weights = torch.randn_like(features)
# and a true bias term
bias = torch.randn((1, 1))

먼저 실습을 위해서 random한 숫자로 이루어진 tensor 데이터를 생성한다.

- `features = torch.randn((1, 5))` : shape이 `(1, 5)`인 tensor를 생성한다. 표준 정규분포(평균이 0이고 표준편차가 1인 정규분포)로부터 random 숫자로 생성된다.

- `weights = torch.randn_like(features)` : `features`와 동일한 shape을 가지는 tensor를 생성한다.

- `bias = torch.randn((1, 1))` : 단일한 숫자를 normal distribution로 부터 생성한다.

PyTorch tensors 는 Numpy arrays 처럼 더하기 빼기 등의 연산이 가능하다. 

> **실습** : input features인 `features`, weights `weights`, 그리고 bias `bias`로 output 데이터 y를 생성해 보자. [`torch.sum()`](https://pytorch.org/docs/stable/torch.html#torch.sum) 또는 `.sum()` method 를 사용할 수 있다.이를 사용하여 다음 식을 구성해 보자. `activation`은 앞에서 생성한 activation function(=sigmoid)를 사용한다.
$$
\begin{align}
y &= f(w_1 x_1 + w_2 x_2 + b) \\
y &= f\left(\sum_i w_i x_i +b \right)
\end{align}
$$

In [4]:
### Solution

# Now, make our labels from our data and true weights

y = activation(torch.sum(features * weights) + bias)
y = activation((features * weights).sum() + bias)

multiplication과 sum 연산은  matrix multiplication으로 구현 가능하다. 일반적으로 matrix multiplications 을 사용하는 것이 연산에 효율적이다.   

그래서 위에 했던 연산을 matrix multiplication으로 바꾸어 보자. 이를 위해 [`torch.mm()`](https://pytorch.org/docs/stable/torch.html#torch.mm) 또는 [`torch.matmul()`](https://pytorch.org/docs/stable/torch.html#torch.matmul) 을 사용한다. 연산하는 두 matrix의 shape이 맞지 않으면 다음과 error가 발생하므로 shape을 주의하자.

```python
>> torch.mm(features, weights)

---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
<ipython-input-13-15d592eb5279> in <module>()
----> 1 torch.mm(features, weights)

RuntimeError: size mismatch, m1: [1 x 5], m2: [1 x 5] at /Users/soumith/minicondabuild3/conda-bld/pytorch_1524590658547/work/aten/src/TH/generic/THTensorMath.c:2033
```

이러한 에러는 텐서의 shape(=dim)정보가 맞지 않기 때문에 발생한다.  

다음과 같은 사항을 반드시 주의하자.
- 첫번째 tensor의 column의 갯수와  두번째 tensor의 row 갯수가 서로 일치해야 한다.
- 만약 `features` 가 `weights` 같은 shape `(1, 5)` 라고 하면 `weights` 의 shape를 matrix multiplication 에 맞도록 바꾸어야 한다..

**Note:** `tensor.shape`로 shape를 확인할 수 있다. 

변경을 위한 옵션 : [`weights.reshape()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.reshape), [`weights.resize_()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.resize_), 또는 [`weights.view()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view).

일반적으로 `.view()` 메소드를 많이 사용한다. 우리는 이 실습에서 `weights`를 5행 1열의 matrix로 변경하여 수행한다. `weights.view(5, 1)`.

> **실습** : 위와 동일하게 y를 matrix multiplication로 구현하시오.

In [5]:
## Solution

y = activation(torch.mm(features, weights.view(5,1)) + bias)

### Stack them up!

앞선 예제에서는 단일 뉴런으로 구성했지만 인공 신경망은 이런 뉴런들을 layer로 여럿 쌓아서 작동하는 알고리즘이다. 첫번째 layer의 output이 두번째 layer의 뉴런 입력이된다. 입력 뉴런과 출력 뉴런의 개수가 많고 layer를 쌓기 위해서 matrix를 사용한다.

![img](../assets/multilayer_diagram_weights.png)

**input layer** : 그림에서 맨 아래쪽의 layer로 입력에 해당  
**hidden layer** : 중간 layer  
**output layer** : 맨 윗쪽의 layer.  

network를 수학적으로  matrix multiplication 으로 표현하면 다음과 같다. 
$$
\vec{h} = [h_1 \, h_2] = 
\begin{bmatrix}
x_1 \, x_2 \cdots \, x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_{11} & w_{12} \\
           w_{21} &w_{22} \\
           \vdots &\vdots \\
           w_{n1} &w_{n2}
\end{bmatrix}
$$

output layer를 간단히 표현 하면 다음과 같다.

$$
y =  f_2 \! \left(\, f_1 \! \left(\vec{x} \, \mathbf{W_1}\right) \mathbf{W_2} \right)
$$

In [6]:
### Generate some data
torch.manual_seed(7) # Set the random seed so things are predictable

# Features are 3 random normal variables
features = torch.randn((1, 3))

# Define the size of each layer in our network
n_input = features.shape[1]     # Number of input units, must match number of input features
n_hidden = 2                    # Number of hidden units 
n_output = 1                    # Number of output units

# Weights for inputs to hidden layer
W1 = torch.randn(n_input, n_hidden)
# Weights for hidden layer to output layer
W2 = torch.randn(n_hidden, n_output)

# and bias terms for hidden and output layers
B1 = torch.randn((1, n_hidden))
B2 = torch.randn((1, n_output))

> **실습** : 위에서 생성한 data를 사용하여 network의 출력값을 계산하고 화면에 출력해 보시오. 

In [7]:
### Solution

h = activation(torch.mm(features, W1) + B1)
output = activation(torch.mm(h, W2) + B2)
print(output)

tensor([[ 0.3171]])


`tensor([[ 0.3171]])`라는 결과가 나오는가?

## Numpy to Torch and back

PyTorch의 tensor와 numpy array는 서로 변환 가능하다.  
- `torch.from_numpy()`: numpy array를 tensor로 
- `.numpy()` method : tensor를 numpy array로.

In [16]:
import numpy as np
a = np.random.rand(4,3)
a

array([[ 0.33669496,  0.59531562,  0.65433944],
       [ 0.86531224,  0.59945364,  0.28043973],
       [ 0.48409303,  0.98357622,  0.33884284],
       [ 0.25591391,  0.51081783,  0.39986403]])

**실습** : a numpy array를 tensor로 변환하시오.

In [17]:
b = torch.from_numpy(a)
b


 0.3367  0.5953  0.6543
 0.8653  0.5995  0.2804
 0.4841  0.9836  0.3388
 0.2559  0.5108  0.3999
[torch.DoubleTensor of size 4x3]

**실습** : 위의 b를 numpy array로 변환해 보시오.

In [18]:
b.numpy()

array([[ 0.33669496,  0.59531562,  0.65433944],
       [ 0.86531224,  0.59945364,  0.28043973],
       [ 0.48409303,  0.98357622,  0.33884284],
       [ 0.25591391,  0.51081783,  0.39986403]])

**실습** : 위의 예제에서 Numpy array와 Torch tensor는 메모리를 공유한다.그래서 in-place을 수행하면 a, b 값이 모두 변하는 것을 확인할 수 있다. 다음을 수행하여 결과를 확인하시오.

In [19]:
# Multiply PyTorch Tensor by 2, in place
b.mul_(2)


 0.6734  1.1906  1.3087
 1.7306  1.1989  0.5609
 0.9682  1.9672  0.6777
 0.5118  1.0216  0.7997
[torch.DoubleTensor of size 4x3]

In [20]:
# Numpy array matches new values from Tensor
a

array([[ 0.67338991,  1.19063124,  1.30867888],
       [ 1.73062448,  1.19890728,  0.56087946],
       [ 0.96818606,  1.96715243,  0.67768568],
       [ 0.51182782,  1.02163565,  0.79972807]])